In [10]:
#only for web cam
import cv2
import mediapipe as mp
import numpy as np


mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)



# for thumb bending
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)
    

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip frame for natural mirror view
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    h, w, _ = frame.shape
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]

            finger_states = {}

            # Thumb joints angles and bend status
            cmc_angle = calculate_angle(landmarks[0], landmarks[1], landmarks[2])
            mcp_angle = calculate_angle(landmarks[1], landmarks[2], landmarks[3])
            ip_angle = calculate_angle(landmarks[2], landmarks[3], landmarks[4])

            finger_states["Thumb"] = ("Bent" if ip_angle < 115 else "Straight", ip_angle)

            # Other fingers bend detection (tip vs PIP y comparison)
            fingers = {
                "Index": [5, 6, 7, 8],
                "Middle": [9, 10, 11, 12],
                "Ring": [13, 14, 15, 16],
                "Pinky": [17, 18, 19, 20]
            }

            for finger, idxs in fingers.items():
                tip_y = landmarks[idxs[3]][1]
                pip_y = landmarks[idxs[1]][1]
                bend_state = "Bent" if tip_y > pip_y else "Straight"

                # Additionally calculate MCP, PIP, DIP angles
                mcp_angle_f = calculate_angle(landmarks[0], landmarks[idxs[0]], landmarks[idxs[1]])
                pip_angle_f = calculate_angle(landmarks[idxs[0]], landmarks[idxs[1]], landmarks[idxs[2]])
                dip_angle_f = calculate_angle(landmarks[idxs[1]], landmarks[idxs[2]], landmarks[idxs[3]])

                # Store both bend state and average finger angle for display
                avg_angle = (mcp_angle_f + pip_angle_f + dip_angle_f) / 3
                finger_states[finger] = (bend_state, avg_angle)

            # Display bend states and angles
            y_pos = 30
            for finger, (state, angle) in finger_states.items():
                color = (0, 255, 0) if state == "Bent" else (0, 0, 255)
                cv2.putText(frame, f"{finger}: {state}, Angle: {angle:.1f}", 
                            (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                y_pos += 30
    else:
        cv2.putText(frame, "No hand detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Finger Bend Detection with Angles", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
#using TCP/IP
import socket
import cv2
import mediapipe as mp
import numpy as np

ESP32_IP = '192.168.1.7'
PORT = 12345

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

cap = cv2.VideoCapture(0)

# Create and connect socket once
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    sock.connect((ESP32_IP, PORT))
    print("Connected to ESP32 TCP server")
except Exception as e:
    print(f"Could not connect to ESP32: {e}")
    exit(1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]

            finger_bends = {}
            ip_angle = calculate_angle(landmarks[2], landmarks[3], landmarks[4])
            finger_bends["Thumb"] = 1 if ip_angle < 115 else 0

            fingers = {
                "Index": [5, 6, 7, 8],
                "Middle": [9, 10, 11, 12],
                "Ring": [13, 14, 15, 16],
                "Pinky": [17, 18, 19, 20]
            }

            for finger, idxs in fingers.items():
                tip_y = landmarks[idxs[3]][1]
                pip_y = landmarks[idxs[1]][1]
                finger_bends[finger] = 1 if tip_y > pip_y else 0

            msg = ''.join(str(finger_bends[f]) for f in ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']) + '\n'

            try:
                sock.sendall(msg.encode())
            except Exception as e:
                print(f"Socket error: {e}")
                break

            for idx, (finger, bent) in enumerate(finger_bends.items()):
                text = f"{finger}: {'Bent' if bent else 'Straight'}"
                cv2.putText(frame, text, (10, 30 + idx * 30), cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (0, 255, 0) if bent else (0, 0, 255), 2)

    cv2.imshow("Finger Bend Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

sock.close()
cap.release()
cv2.destroyAllWindows()


Connected to ESP32 TCP server


In [9]:
#using serial communication
#with display
import serial
import time
import cv2
import mediapipe as mp
import numpy as np

# Initialize serial communication
ser = serial.Serial('COM5', 115200, timeout=1)  # Replace 'COM5' with your COM port
time.sleep(2)  # Wait for ESP32 to initialize serial connection

# MediaPipe setup
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]

            # Determine if fingers bend
            finger_bends = {}

            ip_angle = calculate_angle(landmarks[2], landmarks[3], landmarks[4])
            finger_bends["Thumb"] = 1 if ip_angle < 115 else 0

            fingers = {
                "Index": [5, 6, 7, 8],
                "Middle": [9, 10, 11, 12],
                "Ring": [13, 14, 15, 16],
                "Pinky": [17, 18, 19, 20]
            }

            for finger, idxs in fingers.items():
                tip_y = landmarks[idxs[3]][1]
                pip_y = landmarks[idxs[1]][1]
                finger_bends[finger] = 1 if tip_y > pip_y else 0

            # Compose message string and send via serial
            msg = ''.join(str(finger_bends[f]) for f in ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']) + '\n'
            ser.write(msg.encode())

            print("Sent to ESP32:", msg.strip())

            # Add real-time visual feedback on webcam
            for idx, (finger, bent) in enumerate(finger_bends.items()):
                text = f"{finger}: {'Bent' if bent else 'Straight'}"
                cv2.putText(frame, text, (10, 30 + idx * 30), cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (0, 255, 0) if bent else (0, 0, 255), 2)

    cv2.imshow("Finger Bend Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
ser.close()
cv2.destroyAllWindows()


Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 10000
Sent to ESP32: 10000
Sent to ESP32: 10000
Sent to ESP32: 10000
Sent to ESP32: 10000
Sent to ESP32: 10000
Sent to ESP32: 10000
Sent to ESP32: 10000
Sent to ESP32: 10000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 01000
Sent to ESP32: 01000
Sent to ESP32: 01000
Sent to ESP32: 01000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32: 00000
Sent to ESP32